In [ ]:
import os
import cv2
import ast
import json
import subprocess
from glob import glob
from tqdm.notebook import tqdm
from pprint import pprint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Video

# Introduction

# This notebook is to show image with bboxes and make videos, and to check the jump of bbox number in the sequential video frames.

In [ ]:
# Root of input
INPUT_PATH = '../input/tensorflow-great-barrier-reef'
HEIGHT = 720 # image height
WIDTH  = 1280 # image width

# 📝Input data

In [ ]:
df_train = pd.read_csv(INPUT_PATH + '/train.csv')
display(df_train)
print(df_train.info())

In [ ]:
for video_id in df_train['video_id'].unique():
    print(f'video_id: {video_id}')
    print(f'w   annotations:  {sum(df_train[df_train["video_id"]==video_id]["annotations"] == "[]")}')
    print(f'w/o annotations:  {sum(df_train[df_train["video_id"]==video_id]["annotations"] != "[]")}\n')

In [ ]:
# Change the type of 'annotations' from str to list
df_train['annotations'] = df_train['annotations'].apply(ast.literal_eval) # str -> list
# Add columns of image path and number of bboxes, and the difference.
df_train['image_path'] = INPUT_PATH + '/train_images/video_' + df_train['video_id'].astype(str) + '/' + df_train['video_frame'].astype(str) + ".jpg"
df_train['num_bboxes'] = df_train['annotations'].apply(lambda x: len(x))
df_train['diff_num_bboxes'] = df_train['num_bboxes'].diff().fillna(0).astype(int)
display(df_train)

In [ ]:
# Get indexes with 2 or more diff_num_bboxes
indexes = df_train[abs(df_train['diff_num_bboxes'])>2].index.values
display(df_train.iloc[indexes])

# Plot number of bboxes

Plot number of bboxes and the diff.

In [ ]:
def plot_num_bboxes_and_diff(df, video_id):
    
    df['num_bboxes_lag1'] = df.shift(1)['num_bboxes']
    df_video = df[df['video_id']==video_id]
    
    sequence_start_idx = df_video[df_video['sequence_frame']==0].index
    fig, ax = plt.subplots(1, len(sequence_start_idx), figsize=(len(df_video)/600, 3))
    plt.subplots_adjust(wspace=0.1)
    start_idx = df_video[:1].index[0]
    end_idx = df_video[-1:].index[0]
    for i in range(len(sequence_start_idx-1)):
        
        if i < len(sequence_start_idx)-1:
            width = sequence_start_idx[i+1] - sequence_start_idx[i]
        else: # last sequence
            width = end_idx - sequence_start_idx[i]
            
        # plot #bbox
        df_sequence = df.iloc[sequence_start_idx[i]:sequence_start_idx[i] + width]
        ax[i].plot(df_sequence['video_frame'], df_sequence['num_bboxes'],
                   linewidth=0.5, color='gray', linestyle='--', label='#bbox')
        # plot jump of #bbox
        df_diff_2 = df_sequence[abs(df_sequence['diff_num_bboxes'])==2]
        df_diff_3 = df_sequence[abs(df_sequence['diff_num_bboxes'])>=3]
        ax[i].vlines(df_diff_2['video_frame'], ymin=df_diff_2['num_bboxes_lag1'], ymax=df_diff_2['num_bboxes'],
                     color='orange', alpha=0.5, label='diff = 2')
        ax[i].vlines(df_diff_3['video_frame'], ymin=df_diff_3['num_bboxes_lag1'], ymax=df_diff_3['num_bboxes'],
                     color='red',    alpha=0.5, label='diff >= 3')
            
        # visual setting
        ax[i].set_title(f'{i+1}')
        ax[i].set_position([(sequence_start_idx[i]-start_idx)/len(df_video), 0.05, len(df_sequence)/len(df_video), 0.8])    
        ax[i].set_yticks(np.arange(0,20,5))
        ax[i].set_xlabel('video_frame')
        if width<300:
            ax[i].get_xaxis().set_visible(False)
        ax[i].set_xlim([df_sequence['video_frame'].min()-5, df_sequence['video_frame'].max()])
        ax[i].set_ylim([0,20])
        ax[i].spines["top"].set_linewidth(0)
        ax[i].spines["right"].set_linewidth(0)
        ax[i].spines["bottom"].set_linewidth(1)
        ax[i].grid(axis='y', linestyle = "--")
        ax[i].tick_params(color='w')
        if i>=1:
            ax[i].axes.yaxis.set_ticklabels([])
            # ax[i].spines["left"].set_linewidth(0)

    ax[0].set_ylabel('Number of bboxes')
    ax[i].legend(loc=(0.7, 0.5))
    plt.suptitle(f'video_id: {video_id}', x=0.5, y=1.2, fontsize=15)
    plt.show()

In [ ]:
for i in range(3):
    plot_num_bboxes_and_diff(df=df_train, video_id=i)

# 🐟Sample image

Sampling index = 12637, which has 3 diff. of number of bboxes.

In [ ]:
sample_idx = 12637
sample = df_train.iloc[sample_idx]
print(sample)

In [ ]:
def get_bboxes(annotations):
    """
    annotations: list of annotations
    return: bboxes as [x_min, y_min, x_max, y_max]
    """
    if len(annotations)==0:
        return []
    boxes = pd.DataFrame(annotations, columns=['x', 'y', 'width', 'height']).astype(np.int32).values
    # [x_min, y_min, w, h] -> [x_min, y_min, x_max, y_max]
    boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
    boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
    return boxes   

def plot_img_and_bbox(img_path, anntations):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    fig, ax = plt.subplots(1, 1, figsize=(16,10))
    if len(annotations)>0:
        bboxes = get_bboxes(annotations)
        for i, box in enumerate(bboxes):
            # pur bbox on image
            cv2.rectangle(img,
                          (box[0], box[1]),
                          (box[2], box[3]),
                          color = (255, 0, 0),
                          thickness = 2)
            # numbering
            ax.text(box[0], box[1]-5, i+1, color='red')

    ax.set_axis_off()
    ax.imshow(img)


def zoom_bbox(img_path, annotations):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    bboxes = get_bboxes(annotations)
    
    col = 7 if len(bboxes)>=7 else len(bboxes)
    row = np.ceil(len(bboxes)/7).astype(int) if len(bboxes)>7 else 1
    fig, ax = plt.subplots(row, col, figsize=(col*2, row*3))
    cnt = 0
    for i in range(row):
        
        for j in range(col):
                        
            bbox = bboxes[cnt]
            sliced_img = img[bbox[1]:bbox[3], bbox[0]:bbox[2]]
            
            if row==1:
                ax[j].imshow(sliced_img)
                ax[j].set_title(cnt+1, color='red')
                ax[j].set_axis_off()
            else:
                ax[i,j].imshow(sliced_img)
                ax[i,j].set_title(cnt+1, color='red')
                ax[i,j].set_axis_off()
                
            cnt += 1
            
            if cnt==len(bboxes):
                break
    
        if cnt==len(bboxes):
            break      
            
    plt.show() 

In [ ]:
img_path    = sample['image_path']
annotations = sample['annotations']
print('image_id:', sample['image_id'])
# plot image with bboxes
plot_img_and_bbox(img_path, annotations)
# plot zoom of bboxes
zoom_bbox(img_path, annotations)

# Generate 200-frame video around the image with maximum number of bboxes.
ref. https://www.kaggle.com/bamps53/create-annotated-video#kln-23

In [ ]:
def get_img_with_annotations(img_path, annotations):
    img = cv2.imread(img_path)
    video_id = img_path.split('/')[-2].split('_')[-1]
    frame_id = img_path.split('/')[-1].split('.')[0]
    img_id = video_id + '-' + frame_id
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if len(annotations)>0:
        bboxes = get_bboxes(annotations)
        for i, box in enumerate(bboxes):
            # put bbox
            cv2.rectangle(img,
                          (box[0], box[1]),
                          (box[2], box[3]),
                          color = (0, 0, 255),
                          thickness = 2)
    # put image_id, #bbox
    cv2.putText(img,
                f'image_id: {img_id}, #bbox: {len(annotations)}',
                org = (30, 50), 
                color = (0, 0, 255), 
                fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                fontScale=1.0,
                thickness=3)
    
    return img

def make_video(df, video_id, start_frame, end_frame, fps=15, width=WIDTH, height=HEIGHT):
    '''
    df          : DataFrame
    video_id    : 0, 1, or 2
    start_frame : video_frame at start of video
    num_frame   : video_frame at end of video
    return      : path to video
    '''
    video_path = f'video_{video_id}_{start_frame}_to_{end_frame}.mp4' # video after encode
    tmp_path = 'tmp_' + video_path # video before encode (removed after encode)
    video = cv2.VideoWriter(tmp_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
    
    df = df[df['video_id']==video_id].reset_index(drop=True)
    start_idx = df[df['video_frame']==start_frame].index[0]
    end_idx   = df[df['video_frame']==end_frame].index[0]
    df = df.iloc[start_idx:end_idx]
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        image_path  = row['image_path']
        annotations = row['annotations']
        frame = get_img_with_annotations(image_path, annotations)
        video.write(frame)
    
    video.release()
    
    if os.path.exists(video_path):
        os.remove(video_path)
    
    # encode by ffmpeg command 
    subprocess.run(
        ['ffmpeg', 
         '-i', tmp_path, 
         '-loglevel', 'quiet', 
         '-crf', '18', 
         '-preset', 'veryfast', 
         '-vcodec', 'libx264', 
         video_path]
    )
    os.remove(tmp_path)
    
    return video_path

In [ ]:
video_id    = sample['video_id']
start_frame = sample['video_frame'] - 100 # peek before 100 frames
end_frame   = sample['video_frame'] + 100 # peek after 100 frames
print(f'video_id: {video_id}, video_frame: {start_frame} to {end_frame}')
print('Create video ...')
video_path = make_video(df_train,
                        video_id=video_id,
                        start_frame=start_frame,
                        end_frame=end_frame)

In [ ]:
Video(video_path, width=WIDTH*0.7, height=HEIGHT*0.7)

# The change from id=1-9071 to 9072 (around at 6 sec in this video) is small but the number of bboxes jumps up from 4 to 7 as shown below, so some starfishes are not annotated in id=1-9071.

In [ ]:
def show_jump_of_bbox_num(df, index):
    fig, ax = plt.subplots(1, 2, figsize=(20,12))
    for i, idx in enumerate([index-1, index]):
        img_path    = df.iloc[idx]['image_path']
        annotations = df.iloc[idx]['annotations']
        img = get_img_with_annotations(img_path, annotations)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        ax[i].imshow(img)
        ax[i].set_axis_off()
    plt.show()

In [ ]:
show_jump_of_bbox_num(df_train, sample_idx)

# Since the metric of this commpetition is F2, false positives for these non-annotated starfishes may be some tolerant.
Upvote, if found useful.

# Kindly Upvote, if found useful.